## The Greeks

In mathematical terms, the Greeks are simply the partial derivatives of the option pricing formula. In other words, these are the sensitivities in the movement of the option price relative to the movement of other aspects of the input variables. 

Because the assumptions required for the Black-Scholes model do not hold true in practice, the plain vanilla formula presented above is not usually used for pricing options for the purposes of trading. (In fact if one were to rely on Black-Scholes to manage a portfolio of options, that trader would surely lose money.) It is however, largely used for understanding the sensitivities of the option with respect to it's parameters and to hedge.

- **Delta**. Partial derivative of the option value with respect to the change in the underlying stock price. Delta measures how the underlying option moves with respect to moves in the underlying stock. The formula for delta is different for calls and puts.

  Call delta

  $$
  \frac{\partial V}{\partial S} = N(d_1)
  $$

  Put delta

  $$
  \frac{\partial V}{\partial S} = N(d_1) - 1
  $$

- **Gamma**. Second partial derivative of the option value with respect to the change in the underlying stock price. Gamma measures movements in delta or the convexity in the value of the option with respect to the underlying.

  Gamma (same for call and put)

  $$
  \frac{\partial^2 V}{\partial S^2} = \frac{\phi(d_1)}{S\sigma\sqrt{T - t}}
  $$

- **Vega**. Partial derivative of the option value with respect to the change in the volatility of the underlying. Vega measures how the option price moves with respect to the volatility of the underlying.

  Vega (same for call and put)

  $$
  \frac{\partial V}{\partial \sigma} = S\phi(d_1)\sqrt{T - t}
  $$

- **Theta**. Partial derivative of the option value with respect to the change in time. Theta measures how the value of the option decays as time passes.

  Call theta

  $$
  \frac{\partial V}{\partial t} = -\frac{S\phi(d_1)\sigma}{2\sqrt{T - t}} - rKe^{-r(T - t)}N(d_2)
  $$

  Put theta

  $$
  \frac{\partial V}{\partial t} = -\frac{S\phi(d_1)\sigma}{2\sqrt{T - t}} + rKe^{-r(T - t)}N(-d_2)
  $$

- **Rho**. Partial derivative of the option value with respect to change in the risk-free interest rate. Rho measures how the option value changes as the interest rate changes.

  Call rho

  $$
  \frac{\partial V}{\partial r} = K(T - t)e^{-r(T - t)}N(d_2)
  $$

  Put rho

  $$
  \frac{\partial V}{\partial r} = -K(T - t)e^{-r(T - t)}N(-d_2)
  $$

Where $\phi$

$$
\phi(x) = \frac{e^{-\frac{x^2}{2}}}{\sqrt{2\pi}}
$$

### Import Libraries

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")

### Initial Data

In [3]:
# underlying stock price
S = 45.0
# series of underlying stock prices to demonstrate a payoff profile
S_ = np.arange(35.0, 55.0, 0.01) # strike price
K = 45.0
# time to expiration
t = 164.0 / 365.0
# risk free rate
r = 0.02
# volatility
vol = 0.25
# black scholes prices for demonstrating trades
atm_call_premium = 3.20 
atm_put_premium = 2.79
otm_call_premium = 1.39 
otm_put_premium = 0.92

call_payoff = lambda S, K: np.maximum(S_ - K, 0.0)
put_payoff = lambda S, K: np.maximum(K - S_, 0.0)

In [8]:
def N(z):
    return norm.cdf(z)

# helper function phi
def phi(x):
    pi = 3.1416
    return np.exp(-0.5 * x * x) / (np.sqrt(2.0 * pi))

def gamma(S, K, r, t, vol):
    d1 = (1.0 / (vol * np.sqrt(t))) * (np.log(S / K) + (r + 0.5 * vol ** 2.0) * t)
    return phi(d1) / (S * vol * np.sqrt(t))

def vega(S, K, r, t, vol):
    d1 = (1.0 / (vol * np.sqrt(t))) * (np.log(S / K) + (r + 0.5 * vol ** 2.0) * t)
    return (S * phi(d1) * np.sqrt(t)) / 100.0

# call options
def call_delta(S, K, r, t, vol):
    d1 = (1.0 / (vol * np.sqrt(t))) * (np.log(S / K) + (r + 0.5 * vol ** 2.0) * t)
    return N(d1)

def call_theta(S, K, r, t, vol):
    d1 = (1.0 / (vol * np.sqrt(t))) * (np.log(S / K) + (r + 0.5 * vol ** 2.0) * t)
    d2 = d1 - (vol * np.sqrt(t))
    theta = -((S * phi(d1) * vol) / (2.0 * np.sqrt(t))) - (r * K * np.exp(-r * t) * N(d2))
    return theta / 365.0

def call_rho(S, K, r, t, vol):
    d1 = (1.0 / (vol * np.sqrt(t))) * (np.log(S / K) + (r + 0.5 * vol ** 2.0) * t)
    d2 = d1 - (vol * np.sqrt(t))
    rho = K * t * np.exp(-r * t) * N(d2)
    return rho / 100.0

# put options
def put_delta(S, K, r, t, vol):
    d1 = (1.0 / (vol * np.sqrt(t))) * (np.log(S / K) + (r + 0.5 * vol ** 2.0) * t)
    return N(d1) - 1.0

def put_theta(S, K, r, t, vol):
    d1 = (1.0 / (vol * np.sqrt(t))) * (np.log(S / K) + (r + 0.5 * vol ** 2.0) * t)
    d2 = d1 - (vol * np.sqrt(t))
    theta = -((S * phi(d1) * vol) / (2.0 * np.sqrt(t))) + (r * K * np.exp(-r * t) * N(-d2))
    return theta / 365.0

def put_rho(S, K, r, t, vol):
    d1 = (1.0 / (vol * np.sqrt(t))) * (np.log(S / K) + (r + 0.5 * vol ** 2.0) * t)
    d2 = d1 - (vol * np.sqrt(t))
    rho = -K * t * np.exp(-r * t) * N(-d2)
    return rho / 100.0

# print each of the results
print("Black-Scholes call delta %0.4f" % call_delta(S, K, r, t, vol))
print("Black-Scholes put delta %0.4f" % put_delta(S, K, r, t, vol))
print("Black-Scholes gamma %0.4f" % gamma(S, K, r, t, vol))
print("Black-Scholes vega %0.4f" % vega(S, K, r, t, vol))
print("Black-Scholes call theta %0.4f" % call_theta(S, K, r, t, vol))
print("Black-Scholes put theta %0.4f" % put_theta(S, K, r, t, vol))
print("Black-Scholes call rho %0.4f" % call_rho(S, K, r, t, vol))
print("Black-Scholes put rho %0.4f" % put_rho(S, K, r, t, vol))

Black-Scholes call delta 0.5546
Black-Scholes put delta -0.4454
Black-Scholes gamma 0.0524
Black-Scholes vega 0.1192
Black-Scholes call theta -0.0103
Black-Scholes put theta -0.0078
Black-Scholes call rho 0.0978
Black-Scholes put rho -0.1026


### Interpretation of Black-Scholes Results

#### Delta
- **Call Delta (0.5546)**: This indicates that for a small increase in the underlying stock price, the price of the call option is expected to increase by approximately 0.5546 units. A delta of 0.5546 means the call option is moderately sensitive to changes in the stock price.
- **Put Delta (-0.4454)**: This indicates that for a small increase in the underlying stock price, the price of the put option is expected to decrease by approximately 0.4454 units. A delta of -0.4454 means the put option is moderately sensitive to changes in the stock price, but in the opposite direction to the call option.

#### Gamma
- **Gamma (0.0524)**: A gamma of 0.0524 means that for a small increase in the stock price, the delta of the option will increase by 0.0524. Gamma is the same for both call and put options and measures the convexity of the option's value.

#### Vega
- **Vega (0.1192)**: A vega of 0.1192 means that for a 1% increase in volatility, the price of the option is expected to increase by approximately 0.1192 units. Both call and put options have the same vega.

#### Theta
- **Call Theta (-0.0103)**: This indicates the rate at which the price of the call option declines as time passes, holding all else constant. A theta of -0.0103 means that the call option loses approximately 0.0103 units of its value per day.
- **Put Theta (-0.0078)**: A theta of -0.0078 means that the put option loses approximately 0.0078 units of its value per day. Both call and put options typically have negative theta, reflecting the time decay of options.

#### Rho
- **Call Rho (0.0978)**: A rho of 0.0978 means that for a 1% increase in the interest rate, the price of the call option is expected to increase by approximately 0.0978 units.
- **Put Rho (-0.1026)**: A rho of -0.1026 means that for a 1% increase in the interest rate, the price of the put option is expected to decrease by approximately 0.1026 units. Rho for calls is positive as higher rates increase the call's value, while rho for puts is negative as higher rates decrease the put's value.